In [1]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame

import geopandas as gpd
import pygplates
from gprm import ReconstructionModel
import get_distances_for_dataframe as gdfd


pygmt-session [WARNING]: Representation of font type not recognized. Using default.
pygmt-session [WARNING]: Representation of font type not recognized. Using default.
pygmt-session [WARNING]: Representation of font type not recognized. Using default.
begin [WARNING]: Representation of font type not recognized. Using default.
begin [WARNING]: Representation of font type not recognized. Using default.
begin [WARNING]: Representation of font type not recognized. Using default.


In [2]:
from gprm.datasets import Reconstructions
Y2019 = Reconstructions.fetch_Young2019()
Y2019

Name: Young++2019
Rotation Files:
   - Global_410-250Ma_Young_et_al.rot
   - Global_250-0Ma_Young_et_al.rot
Static Polygon Files:
   - Global_GPlates_PresentDay_StaticPlatePolygons_Young_et_al.shp
Coastlines Files:
   - Global_coastlines_Young_et_al_low_res.shp
Continent Polygon Files:
   - PresentDay_ContinentalPolygons_Young_et_al.shp
Dynamic Polygon Files:
   - Global_Mesozoic-Cenozoic_plate_boundaries_Young_et_al.gpml
   - Global_Paleozoic_plate_boundaries_Young_et_al.gpml
   - TopologyBuildingBlocks_Young_et_al.gpml

In [3]:
# This cell already output a file, so we don't need run it if it doesn't change.
# If you want change it, only need convert makedown into code, then it can run normally. 

anchor_plate_id = 0
df_ABC = pd.read_excel('data/DZDB3.xlsx',sheet_name="Samples")
df_ABC['FROMAGE'] = '0'
df_ABC['TOAGE'] = '0'
for i in range(len(df_ABC['FROMAGE'])):
    df_ABC.loc[i,('FROMAGE')] = df_ABC.loc[i,('Est. Depos. Age (Ma)')] + 4.99
    df_ABC.loc[i,('TOAGE')] = df_ABC.loc[i,('Est. Depos. Age (Ma)')] - 5.
    
Sample_Key = df_ABC['Ref_Sample_Key']
Sample_Key = Sample_Key.to_list()

distance_subduction = []
for key_i in Sample_Key:
    df_ABC_key = df_ABC[df_ABC['Ref_Sample_Key'] == key_i]
    reconstruction_time = float(np.round(df_ABC_key['Est. Depos. Age (Ma)']))
    rlo,rla,rd = gdfd.get_distances_for_dataframe(df_ABC_key, Y2019, reconstruction_time,anchor_plate_id=anchor_plate_id, remove_points_older_than_polygon=True)

    if len(rd)>0.:
        df_ABC.loc[df_ABC['Ref_Sample_Key'] == key_i,'distance_subduction'] = rd
    else:
        df_ABC.loc[df_ABC['Ref_Sample_Key'] == key_i,'distance_subduction'] = 'points_older_than_polygon'
    


In [4]:
df_Sample_60 = df_ABC
df_Data_detrital = pd.read_excel('data/DZDB3.xlsx',sheet_name="UPb_Data")
df_Ref_detrital = pd.read_excel('data/DZDB3.xlsx',sheet_name="References")


In [5]:
with pd.ExcelWriter(r"data/DZDB3.xlsx") as writer:
    df_Ref_detrital.to_excel(writer,sheet_name='References',index = False)
    df_Sample_60.to_excel(writer,sheet_name='Samples',index = False)
    df_Data_detrital.to_excel(writer,sheet_name='UPb_Data',index = False)

In [7]:
len(df_Sample_60)
df_Sample_60['type'].value_counts()


B    2400
A    2015
C    1985
Name: type, dtype: int64